#  Jogo dos Rastros
## Projeto nº 2
### Introdução à Inteligência Artificial edição 2020/21


## Grupo: XX

### Elementos do Grupo

Nome:

Número:

Nome:

Número

Nome:

Número

### Explicação da função e sua lógica
(descrição)

In [115]:
from rastros import *

def fun_aval_42(state, player):
    win_or_loss = state.compute_utility(player)
    if win_or_loss != 0:
        return win_or_loss * 3
    else:
        goal = (8, 1) if player == "S" else (1, 8)
        score_white_goal = 1.0 / (2 ** distancia(state.white, goal))
        sum_score_moves = 0
        for move in state.moves():
            score_move_goal = 1.0 / (2 ** distancia(move, goal))
            sum_score_moves += score_move_goal
        
        return (sum_score_moves * 0.5) + (score_white_goal * 1.5)
    

###########################################################################################
### APAGAR NO FIM #########################################################################
def pergunta_print(game, state):
    state.display()
    print("ultima pos: ", list(state.blacks)[-1])
    if len(state.blacks) <= 1:
        pergunta_print.tst = 0
    pergunta_print.tst = pergunta_print.tst + 1
    print(pergunta_print.tst)
    print("Jogadas possíveis: ", state.moves())
    return eval(input(state.to_move+", para onde quer jogar? "))
    
humano_print = Jogador("print1", pergunta_print)
basilio_test = Jogador("Basilio_test",
                  lambda game, state:
                  alphabeta_cutoff_search_new(state,game,depth_for_all,eval_fn=f_aval_basico))
###########################################################################################
###########################################################################################

### Demonstração
(Mostrem que o código está a funcionar, com exemplos adequados, executem um ou mais jogos, mostrem os seus resultados, exibam situações de jogo que demonstrem que a lógica da vossa função está correctamente implementada.)

In [116]:
jogador42 = Jogador("jogador42",
                  lambda game, state:
                  alphabeta_cutoff_search_new(state,game,depth_for_all,eval_fn=fun_aval_42))

#jg = jogaRastros11(basilio,test1)
#print(jg)

def n_faz_campeonato(num_campeonatos, listaJogadores, nsec=10):
    jogador_vitorias = {}
    print("Progresso com " + str(num_campeonatos) + " campeonatos: ", end='')
    for n_c in range(num_campeonatos):
        campeonato = jogaRastrosNN(listaJogadores, listaJogadores, nsec)
        resultado_jogos = [(a,b,n) for (a,b,(x,n)) in campeonato]
        tabela = dict([(jog.nome, 0) for jog in listaJogadores])
        for jogo in resultado_jogos:
            if jogo[2] == 1:
                tabela[jogo[0]] += 1
            else:
                tabela[jogo[1]] += 1
        classificacao = list(tabela.items())
        classificacao.sort(key=lambda p: -p[1])
        for jog in classificacao:
            jogador_vitorias[jog[0]] = jogador_vitorias.get(jog[0],0) + jog[1]
        print(str(n_c + 1) + " ", end='')
    print("\nJOGADOR", "VITÓRIAS")
    for jogador in jogador_vitorias:
        print('{:11}'.format(jogador), '{:>4}'.format(jogador_vitorias[jogador]))

n_faz_campeonato(25, [jogador42, basilio])

Progresso com 25 campeonatos: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 
JOGADOR VITÓRIAS
jogador42     21
Basilio       29
